# STOP Ransomware Configuration Extractor
This config extractor was tested on the following samples (sha256):
- 236259fb27568c5b6ba0ed090909d2f1aeb70258673f3b561514350a65eba77a
- c73cb93ad9ad6d003b505ce2b960d75467ad612786e0559c74dca18426fb9400
- 4ad7b8d228fe32d82b0373ce886f224f47c2e06a59d394c634160c70083b5f32
- 18f3120c547c3568ab79e3762b4f3112f6f8c5024c35e6f3a2b43242191ce91d

In [17]:
import pefile
import re

file_path = r'path_to_executable'

def extract_section(file_path):
    file = open(file_path, 'rb').read()
    pe = pefile.PE(data=file)
    for section in pe.sections:
         if '.rdata' in section.Name.decode(encoding='utf-8').rstrip('x00'):
             rdata_start_address = section.VirtualAddress
             return (section.get_data(section.VirtualAddress, section.SizeOfRawData))

def config_offset(encrypted_data_addr, section_start):
    data_offset = int(section_start, 16) - int(encrypted_data_addr, 16)
    return data_offset

def xor_decrypt(data, key):
    result = bytearray()
    for byte in data:
        result.append(byte ^ key)
    return bytes(result)

def remove_hex_bytes(byte_array):
    clean_data = ""
    for byte in byte_array:
        # Filter out non-ASCII characters
        if byte >= 32 and byte <= 126 or byte == 10:
            clean_data += chr(byte)
    return clean_data


rdata_section = extract_section(file_path)

# This is where the encrypted data can be found in the .rdata section
enc_start = config_offset('0x004cc000', '0x00500498') # 0x00500498 is the start address of the encrypted data
enc_end = config_offset('0x004cc000', '0x00505a10')   # 0x00505a10 is the end address of the encrypted data

enc_data = rdata_section[enc_start:enc_end]

key = 0x80    # XOR key
result = xor_decrypt(enc_data, key)

clean_data = remove_hex_bytes(result)


In [18]:
# Command and Control
c2 = []
for i in re.finditer(rb'http?://[a-zA-Z0-9.:/]{2,}', result):
    c2.append(i.group().decode('utf-8'))
print("\033[1m C2: \033[0m", set(c2))


# Ransomware Extension
extension = re.findall(r'[.][a-z]{4}\b', clean_data)
for ext in extension:
    print("\n \033[1m Extension: \033[0m", ext)


# Public Key
key = re.findall(r'(?s)-----BEGIN PUBLIC KEY-----.*?-----END PUBLIC KEY-----', clean_data)
for k in key:
    print("\n \033[1m Public Key: \033[0m \n", k)


# Ransom Note
note = re.findall(r'(?s)ATTENTION.*?Your personal ID:', clean_data)
for n in note:
    print('\n\n \033[1m Ransom Note: \033[0m \n\n', n)


# Personal ID
id = []
for i in re.finditer(rb'[a-zA-Z0-9]{40}', result):
    id.append(i.group().decode('utf-8'))
print("\n \033[1m ID: \033[0m", id[-1:])


 C2:  {'http://brusuax.com/dl/build2.exe', 'http://habrafa.com/files/1/build3.exe', 'http://habrafa.com/test1/get.php'}

  Extension:  .cdpo

  Public Key:  
 -----BEGIN PUBLIC KEY-----\\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAr0GLq\/mHXqBal7ucID+c\\ncFScc8\/W1OCT0GTCVzJgxoMkEGJZqNselbUtChT0yNq5BFkt44IkH\/PtixxLTHYo\\nEmqbXelDPaljN+zsslk8klrc0Or3AWuqZ\/1RR7QbtctpTFJ9hiD0FP1pE4MjvbXb\\nf7KvBkAnDRnPXWcZpw7NvDbt1XOa2cptFcsH7lOLWhFuskkOcE15lQ485pNSOkcm\\nVXpam\/yhHl5jrHs4ue0uSBXJ1yeJqQT\/312wZma6ho0sDTHqC3\/+4+Md8LF8p0OD\\n\/bmb+kMQv0Oaou31VupP4C2qlkjZLaPqzjX\/amkyo8iQ\/1bRZQW5ZbtbXFlFb\/4M\\n\/wIDAQAB\\n-----END PUBLIC KEY-----


  Ransom Note:  

 ATTENTION!

Don't worry, you can return all your files!
All your files like pictures, databases, documents and other important are encrypted with strongest encryption and unique key.
The only method of recovering files is to purchase decrypt tool and unique key for you.
This software will decrypt all your encrypted files.
What guarantees y